In [1]:
import pandas as pd

from SMILESX import main, inference
%matplotlib inline

Using TensorFlow backend.


#### Definition of data

In [2]:
validation_data_dir = "./validation_data/"

In [3]:
extension = '.csv'

In [4]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [5]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [6]:
sol_data = pd.read_csv(validation_data_dir+data_filename+extension)

In [7]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


In [8]:
sol_data = sol_data[['smiles',prop_tag]]

In [9]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [10]:
sol_data.shape

(642, 2)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [11]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 
dalpha_range = [float(ialpha/10.) for ialpha in range(20,40,1)] # Adam's learning rate = 10^(-dalpha_range)

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'lrate', 'type': 'discrete', 'domain': dalpha_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': (1024, 1024)}, # fixed
        {'name': 'lrate', 'type': 'discrete', 'domain': (3, 3)} # fixed
    ]

In [12]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 3, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_epochs = 1,
          bayopt_n_rounds = 1, 
          bayopt_it_factor = 1, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          n_gpus = 1, 
          bridge_type = 'NVLink', 
          batch_size_ref = 8,
          alpha_ref = 22, 
          patience = 50, 
          n_epochs = 100)

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Sampling and splitting of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.80/0.10/0.10


***Data augmentation to True***

Enumerated SMILES:
	Training set: 4285
	Validation set: 501
	Test set: 569

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', 'Cl', ' '], [' ', 'Cl', 'C', ' '], [' ', 'C', 'C', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ' '], [' ', 'C', '(', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ')', 'C', ' '], [' ', 'c', '1', '(', 'C', 'C', ')', 'c', 'c', 'n', 'c', 'c', '1', ' ']]

Number of tokens only present in a training set: 32

Number of tokens only present in a validation set: 27
Is the validation set a subset of the training set: True
What are the tokens by which they differ: set()

Number of tokens only present in a test set: 27
Is the test set a subset o

Epoch 21/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0083 - mean_absolute_error: 0.0699 - mean_squared_error: 0.0083 - val_loss: 0.0110 - val_mean_absolute_error: 0.0795 - val_mean_squared_error: 0.0110
Epoch 22/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0085 - mean_absolute_error: 0.0698 - mean_squared_error: 0.0085 - val_loss: 0.0090 - val_mean_absolute_error: 0.0713 - val_mean_squared_error: 0.0090
Epoch 23/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0090 - mean_absolute_error: 0.0721 - mean_squared_error: 0.0090 - val_loss: 0.0090 - val_mean_absolute_error: 0.0734 - val_mean_squared_error: 0.0090
Epoch 24/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0087 - mean_absolute_error: 0.0717 - mean_squared_error: 0.0087 - val_loss: 0.0089 - val_mean_absolute_error: 0.0699 - val_mean_squared_error: 0.0089
Epoch 25/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0083 - mea

Epoch 57/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0051 - mean_absolute_error: 0.0528 - mean_squared_error: 0.0051 - val_loss: 0.0100 - val_mean_absolute_error: 0.0734 - val_mean_squared_error: 0.0100
Epoch 58/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0055 - mean_absolute_error: 0.0546 - mean_squared_error: 0.0055 - val_loss: 0.0082 - val_mean_absolute_error: 0.0681 - val_mean_squared_error: 0.0082
Epoch 59/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0053 - mean_absolute_error: 0.0550 - mean_squared_error: 0.0053 - val_loss: 0.0102 - val_mean_absolute_error: 0.0726 - val_mean_squared_error: 0.0102
Epoch 60/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0048 - mean_absolute_error: 0.0518 - mean_squared_error: 0.0048 - val_loss: 0.0086 - val_mean_absolute_error: 0.0656 - val_mean_squared_error: 0.0086
Epoch 61/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0052 - mea

Epoch 93/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0036 - mean_absolute_error: 0.0442 - mean_squared_error: 0.0036 - val_loss: 0.0080 - val_mean_absolute_error: 0.0636 - val_mean_squared_error: 0.0080
Epoch 94/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0033 - mean_absolute_error: 0.0427 - mean_squared_error: 0.0033 - val_loss: 0.0079 - val_mean_absolute_error: 0.0636 - val_mean_squared_error: 0.0079
Epoch 95/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0040 - mean_absolute_error: 0.0470 - mean_squared_error: 0.0040 - val_loss: 0.0080 - val_mean_absolute_error: 0.0643 - val_mean_squared_error: 0.0080
Epoch 96/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0034 - mean_absolute_error: 0.0432 - mean_squared_error: 0.0034 - val_loss: 0.0075 - val_mean_absolute_error: 0.0619 - val_mean_squared_error: 0.0075
Epoch 97/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0033 - mea

67/67 [==============================] - 0s 7ms/step - loss: 0.0117 - mean_absolute_error: 0.0838 - mean_squared_error: 0.0117 - val_loss: 0.0195 - val_mean_absolute_error: 0.1104 - val_mean_squared_error: 0.0195
Epoch 49/100
67/67 [==============================] - 1s 7ms/step - loss: 0.0124 - mean_absolute_error: 0.0874 - mean_squared_error: 0.0124 - val_loss: 0.0185 - val_mean_absolute_error: 0.1033 - val_mean_squared_error: 0.0185
Epoch 50/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0122 - mean_absolute_error: 0.0870 - mean_squared_error: 0.0122 - val_loss: 0.0228 - val_mean_absolute_error: 0.1172 - val_mean_squared_error: 0.0228
Epoch 51/100
67/67 [==============================] - 1s 9ms/step - loss: 0.0125 - mean_absolute_error: 0.0865 - mean_squared_error: 0.0125 - val_loss: 0.0238 - val_mean_absolute_error: 0.1244 - val_mean_squared_error: 0.0238
Epoch 52/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0117 - mean_absolute_error: 0.085

273/273 [==============================] - 2s 7ms/step - loss: 0.0234 - mean_absolute_error: 0.1215 - mean_squared_error: 0.0234 - val_loss: 0.0241 - val_mean_absolute_error: 0.1259 - val_mean_squared_error: 0.0241
Epoch 6/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0198 - mean_absolute_error: 0.1107 - mean_squared_error: 0.0198 - val_loss: 0.0193 - val_mean_absolute_error: 0.1078 - val_mean_squared_error: 0.0193
Epoch 7/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0175 - mean_absolute_error: 0.1044 - mean_squared_error: 0.0175 - val_loss: 0.0201 - val_mean_absolute_error: 0.1016 - val_mean_squared_error: 0.0201
Epoch 8/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0166 - mean_absolute_error: 0.0977 - mean_squared_error: 0.0166 - val_loss: 0.0157 - val_mean_absolute_error: 0.0958 - val_mean_squared_error: 0.0157
Epoch 9/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0143 - mean_absolute_error:

273/273 [==============================] - 2s 7ms/step - loss: 0.0032 - mean_absolute_error: 0.0432 - mean_squared_error: 0.0032 - val_loss: 0.0099 - val_mean_absolute_error: 0.0653 - val_mean_squared_error: 0.0099
Epoch 42/100
273/273 [==============================] - 3s 9ms/step - loss: 0.0030 - mean_absolute_error: 0.0415 - mean_squared_error: 0.0030 - val_loss: 0.0118 - val_mean_absolute_error: 0.0693 - val_mean_squared_error: 0.0118
Epoch 43/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0035 - mean_absolute_error: 0.0444 - mean_squared_error: 0.0035 - val_loss: 0.0088 - val_mean_absolute_error: 0.0608 - val_mean_squared_error: 0.0088
Epoch 44/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0029 - mean_absolute_error: 0.0409 - mean_squared_error: 0.0029 - val_loss: 0.0101 - val_mean_absolute_error: 0.0625 - val_mean_squared_error: 0.0101
Epoch 45/100
273/273 [==============================] - 2s 9ms/step - loss: 0.0028 - mean_absolute_er

Epoch 77/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0014 - mean_absolute_error: 0.0282 - mean_squared_error: 0.0014 - val_loss: 0.0093 - val_mean_absolute_error: 0.0610 - val_mean_squared_error: 0.0093
Epoch 78/100
273/273 [==============================] - 2s 9ms/step - loss: 0.0015 - mean_absolute_error: 0.0288 - mean_squared_error: 0.0015 - val_loss: 0.0096 - val_mean_absolute_error: 0.0643 - val_mean_squared_error: 0.0096
Epoch 79/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0014 - mean_absolute_error: 0.0290 - mean_squared_error: 0.0014 - val_loss: 0.0087 - val_mean_absolute_error: 0.0560 - val_mean_squared_error: 0.0087
Epoch 80/100
273/273 [==============================] - 3s 11ms/step - loss: 0.0019 - mean_absolute_error: 0.0320 - mean_squared_error: 0.0019 - val_loss: 0.0086 - val_mean_absolute_error: 0.0562 - val_mean_squared_error: 0.0086
Epoch 81/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0017 - me

In [13]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

***SMILES_X for inference starts...***


***Checking the SMILES list for inference***

***Data augmentation.***

Enumerated SMILES: 5

***Tokenization of SMILES.***

Full vocabulary: ['pad', 'unk', 'Cl', '[N+]', '\\', ')', '2', '-', 'c', '1', 'P', 'n', '=', ' ', '[C@@]', 'S', '3', 'Br', '#', '/', '[O-]', '[C@@H]', '[C@H]', '[nH]', '5', '4', 'C', 's', '(', 'N', 'F', '[S+2]', 'I', '[C@]', 'O']
Of size: 35

Maximum length of tokenized SMILES: 51 tokens

***Inference of SMILES property done.***


In [14]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.406713,0.0213377
1,CCC,0.43053,0.020964
2,C=O,0.00962523,0.0223681
